In [1]:
from bs4 import BeautifulSoup as bs
import time
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [2]:
driver = webdriver.Chrome('/usr/bin/chromedriver')

In [3]:
all_data = []
for pages in range(1,4):
    """
    it was discovered that for each page,only one part of the url is changing, i am going to modifiying the url using f string
    one can change pages by clicking the pagination at the bottom 
    """
    url = f'https://hfr.health.gov.ng/facilities/hospitals-search?_token=4Wll44OzOr1kZWrvOzm7FGC1y3zCYbgGs99vHRSf&state_id=124&ward_id=0&facility_level_id=0&ownership_id=0&operational_status_id=1&registration_status_id=2&license_status_id=1&geo_codes=0&service_type=0&service_category_id=0&entries_per_page=20&page={pages}'
    driver.get(url)
    time.sleep(5)
    
    #gets the source code of the table
    hosp = driver.find_element_by_id('hosp').get_attribute('innerHTML')
    
    #parse the code
    soup = bs(hosp,'lxml')
    
    #find all the rows
    rows = soup.find_all('tr')
    
    #the first element is the cloumns while the others are rows
    row = rows[1:]
    
    for j,i in enumerate(row):
        details = [a.text for a in i.find_all('td')[:-1]]
        Facility_UID = details[3]
        xpath = f'//*[@id="hosp"]/tbody/tr[{j+1}]/td[9]/a/button'
        #just in case it didn't close the pop up the first time
        webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
        time.sleep(5)
        temp_all_data = [[Facility_UID]]
        #sometimes, the click does not work the first time and fails twice too, that is while i used for loop
        for i in range(5):
            try:
                driver.find_element_by_xpath(xpath).click()
            except:
                pass



        html = driver.find_element_by_xpath('//*[@id="view_details"]/div').get_attribute('innerHTML')
        soup = bs(html,'lxml')   
        webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
        panel = soup.find_all('div',{'class':'panel panel-default'})

        for k in panel:
            each_row = k.find_all('div',{'class':'row'})
            data = []
            for l in each_row:
                data.append(l.find('div').text)
            temp_all_data.append(data)
        all_data.append(temp_all_data)

In [4]:
len(all_data)

60

In [5]:
#flatten the list
flattened_data = []
for data in all_data:
    flattened = [val for sub_data in data for val in sub_data]
    flattened_data.append(flattened)

In [6]:
length_data = []
for i in flattened_data:
    length_data.append(len(i))

In [7]:
panel = soup.find_all('div',{'class':'panel panel-default'})

labels = ['Facility_uid']
for k in panel:
    each_row = [i.text for i in k.find_all('label')]
    for l in each_row:
        labels.append(l)
labels

['Facility_uid',
 'Facility Code:',
 'State Unique ID:',
 'Registration No:',
 'Facility Name:',
 'Alternate Name:',
 'Start Date:',
 'Ownership:',
 'Ownership Type:',
 'Facility Level:',
 'Facility Level Option:',
 'Days of Operation:',
 'Hours of Operation:',
 'State:',
 'LGA:',
 'Ward:',
 ' Physical Location:',
 'Postal Address:',
 'Longitude:',
 'Latitude:',
 'Phone Number:',
 'Alternate Number:',
 'Email Address:',
 'Website:',
 'Operational Status:',
 'Registration Status:',
 'License Status:',
 'Out Patient Services:',
 'In Patient Services:',
 'Medical Services:',
 'Surgical Services:',
 'Obsterics and Gynecology Services:',
 'Pediatrics Services:',
 'Dental Services:',
 'Specific Clinical Services:',
 'Total number of Beds:',
 'Onsite Laboratory:',
 'Onsite Imaging:',
 'Onsite Pharmacy:',
 'Mortuary Services:',
 'Ambulance Services:',
 'Number of Doctors:',
 'Number of Pharmacists:',
 'Number Pharmacy Technicians:',
 'Number of Dentists:',
 'Number of Dental Technicians:',
 'N

In [8]:
df = pd.DataFrame(data=flattened_data,columns=labels)

In [13]:
df

,Facility_uid,Facility Code:,State Unique ID:,Registration No:,Facility Name:,Alternate Name:,Start Date:,Ownership:,Ownership Type:,Facility Level:,...,Number of Midwifes:,Number of Nurses/Midwifes:,Number of Lab Technicians:,Number of Lab Scientits:,Health Records/HIM Officers:,Number of Community Health Officer:,Number of Community Health Extension Worker:,Number of Junior Com Health Extension Worker:,Number of Environmental Health Officers:,Number of Health Attendant/Assistant:
0,51951895,24/01/1/1/2/0003,24011020002,,Adu Medical Clinic,,1987-12-01,Private,For Profit,Primary,...,1,,,,,,2,,,4
1,30605129,24/01/1/1/2/0011,2401120005,,Habibat Memorial Convalescent Home,,2004-01-01,Private,For Profit,Primary,...,,1,,,,1,,,,4
2,30710800,24/01/1/1/2/0011,2401120005,,Habibat Memorial Convalescent Home,,2004-01-01,Private,For Profit,Primary,...,,1,,,,1,,,,4
3,21456488,24/01/1/2/2/0030,2401120079,,Bolasad Hospital,,2017-01-11,Private,For Profit,Primary,...,,1,,1,,,,,,2
4,72877733,24/01/1/2/2/0030,2401120079,,Bolasad Hospital,,2017-01-11,Private,For Profit,Primary,...,,1,,1,,,,,,2
5,81233574,24/01/1/2/2/0007,2401120057,,Glorious Favour Nursing Home,,2013-01-01,Private,For Profit,Primary,...,,1,,,,,,,,1
6,86530408,24/01/1/2/2/0007,2401120057,,Glorious Favour Nursing Home,,2013-01-01,Private,For Profit,Primary,...,,1,,,,,,,,1
7,38579933,24/01/1/2/2/0010,2401120081,,Keeb Ideal Surgical Clinic,,2015-01-02,Private,For Profit,Primary,...,,1,,,,,,,,2
8,80710017,24/01/1/2/2/0004,2401120081,,Krisopath Hospital,,2017-01-01,Private,For Profit,Primary,...,,,,,,,,,,4
9,63533294,24/01/1/2/2/0004,2401120081,,Krisopath Hospital,,2017-01-01,Private,For Profit,Primary,...,,,,,,,,,,4
